# Fairness statistics for adult income

## Task 1
<!-- 
$Y$ - individual will use XAI

$\hat{Y}$ - individual enrolled in training

#### Demographic parity

$$P(\hat{Y}|Red) = 0.5$$
$$P(\hat{Y}|Blue) = 0.65$$
$$P(\hat{Y}|Red) \ne P(\hat{Y}|Blue)$$
No demographic parity

#### Equal opportunity

$$P(\hat{y}|Red, Y=1) = 0.5$$
$$P(\hat{y}|Blue, Y=1) = 0.75$$
Coefficient: $0.75/0.5=1.5$

#### Predictive rate parity
$$P(Y=1|Red, \hat{Y}=1) = 1/2$$
$$P(Y=1|Blue, \hat{Y}=1) = 60/65 = 12/13$$
Coefficient: $\frac{12/13}{1/2}=24/13 \approx 1.85$ -->


![](imgs/task_1.png)

## Task 2

Dataset used is The Adult income dataset
([source](https://www.kaggle.com/datasets/wenruliu/adult-income-dataset)).


I have preprocessed the dataset by one hot encoding categorical features.

I have selected the protected variable `race` with the privileged group = `White`. For the following models:
- standard Random Forest
- Logistic Regression
- Random Forest trained without the privileged group and with reweighting

I will show the following fairness coefficients:
- Statistical parity
- Equal opportunity
- Predictive parity.

### Fairness coefficients for standard Random Forest

![](imgs/rf.png)

We can see that for Random Forest model for the selected protected variable `race` and the privileged group `White`
equal opportunity and predictive parity ratio follow the four-fifth rule. Only in terms of the standard parity the model
is not fair (from the 3 selected coefficients). As model tries to best fit to the dataset it is expected as in the
dataset in the privileged group higher percentage of subjects have income higher than 50K.

![](imgs/lr.png)
Fairness coefficients for the Logistic Regression model follow very similar patter to those of Random Forest model.
Statistical parity is slightly closer to 1 and Predictive parity is less fair but the differences are negligible.

![](imgs/rf_reweight.png)
I have trained the Random Forest model without the protected variable and with reweight mitigation. None of the fairness
coefficient shows better scores compared to the standard Random Forest model. This suggests high correlation between
variables in the dataset and model is able to easily adjust for the lack of the variable.

## Appendix

### Install required packages.

In [1]:
%%capture
%pip install dalex jinja2 kaleido numpy nbformat pandas plotly scikit-learn

### Imports and loading dataset

In [2]:
import dalex as dx
import numpy as np
import pandas as pd
import plotly.express as px
from copy import copy
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

rng = np.random.default_rng(0)

TARGET_COLUMN = "income"
df = pd.read_csv("adult.csv")
df.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K


Shuffling the data, extracting target column and one hot encoding categorical columns.

In [3]:
df = df.sample(frac=1, random_state=0).reset_index(drop=True)

y = df[[TARGET_COLUMN]]
y = y == ">50K"
y = y.rename(columns={"income": "income>50K"})

x = df.drop(TARGET_COLUMN, axis=1)

categorical_cols = ["workclass", "education", "marital-status", "occupation", "relationship", "race", "gender", "native-country"]
numerical_cols = list(set(x.columns) - set(categorical_cols))

x = pd.get_dummies(x, columns=categorical_cols, drop_first=True)
n_columns = len(x.columns)

categorical_cols, numerical_cols

(['workclass',
  'education',
  'marital-status',
  'occupation',
  'relationship',
  'race',
  'gender',
  'native-country'],
 ['age',
  'fnlwgt',
  'educational-num',
  'hours-per-week',
  'capital-loss',
  'capital-gain'])

In [4]:
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.75, random_state=0, shuffle=True, stratify=y)

In [5]:
protected_variable = x_test.race_White.apply(lambda x: "White" if x else "Non-white")
privileged_group = "White"

## Random Forest model

In [6]:
def plot_fobject(model, predict_func, plot_filename):
    explainer = dx.Explainer(model, x_test, y_test, predict_function=predict_func)

    fobject = explainer.model_fairness(
        protected=protected_variable,
        privileged=privileged_group
    )

    print(fobject.fairness_check())

    fig = fobject.plot(show=False).update_layout(
        autosize=False,
        width=900,
        height=450)
    fig.write_image(f"imgs/{plot_filename}")
    fig.show()
    

In [7]:
def predict_random_forest(m, d): 
    return m.predict_proba(d)[:, 1]

model = RandomForestClassifier(random_state=0).fit(x_train, y_train)
plot_fobject(model, predict_random_forest, "rf.png")

/tmp/ipykernel_13226/3157806445.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model = RandomForestClassifier(random_state=0).fit(x_train, y_train)


Preparation of a new explainer is initiated

  -> data              : 12211 rows 100 cols
  -> target variable   : Parameter 'y' was a pandas.DataFrame. Converted to a numpy.ndarray.
  -> target variable   : 12211 values
  -> model_class       : sklearn.ensemble._forest.RandomForestClassifier (default)
  -> label             : Not specified, model's class short name will be used. (default)
  -> predict function  : <function predict_random_forest at 0x7fbbf8e45550> will be used
  -> predict function  : Accepts pandas.DataFrame and numpy.ndarray.


/home/krystian/PythonProjects/eXplainableMachineLearning-2023/.venv/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


  -> predicted values  : min = 0.0, mean = 0.243, max = 1.0
  -> model type        : classification will be used (default)
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = -1.0, mean = -0.00383, max = 1.0
  -> model_info        : package sklearn

A new explainer has been created!
Bias detected in 2 metrics: FPR, STP

Conclusion: your model is not fair because 2 or more criteria exceeded acceptable limits set by epsilon.

Ratios of metrics, based on 'White'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
                TPR       ACC       PPV      FPR       STP
Non-white  0.862992  1.061538  0.988889  0.47619  0.526786
None


## Logistic Regression

In [8]:
def lr_predict_func(m, d):
    pred = m.decision_function(d)
    return 1 / (1 + np.exp(-pred))

lr_clf = RidgeClassifier(random_state=0).fit(x_train, y_train.squeeze())

plot_fobject(lr_clf, lr_predict_func, "lr.png")

Preparation of a new explainer is initiated

  -> data              : 12211 rows 100 cols
  -> target variable   : Parameter 'y' was a pandas.DataFrame. Converted to a numpy.ndarray.
  -> target variable   : 12211 values
  -> model_class       : sklearn.linear_model._ridge.RidgeClassifier (default)
  -> label             : Not specified, model's class short name will be used. (default)
  -> predict function  : <function lr_predict_func at 0x7fbc45092280> will be used
  -> predict function  : Accepts pandas.DataFrame and numpy.ndarray.
  -> predicted values  : min = 0.157, mean = 0.379, max = 0.924
  -> model type        : classification will be used (default)
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = -0.769, mean = -0.14, max = 0.767
  -> model_info        : package sklearn

A new explainer has been created!
Bias detected in 2 metrics: FPR, STP

Conclusion: your model is not fair because 2 or more criteria exceeded acceptable limits

/home/krystian/PythonProjects/eXplainableMachineLearning-2023/.venv/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning:

X does not have valid feature names, but RidgeClassifier was fitted with feature names



In [9]:
x_train_without_prot, x_test_without_prot = x_train.drop("race_White", axis=1), x_test.drop("race_White", axis=1)

model_without_prot = RandomForestClassifier(random_state=0).fit(x_train_without_prot, y_train)

/tmp/ipykernel_13226/2203245347.py:3: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



In [10]:
explainer_without_prot = dx.Explainer(
    model_without_prot, 
    x_test_without_prot, 
    y_test,
    predict_function=predict_random_forest,
    label="Random Forest without the protected attribute",
    verbose=False
)

fobject_without_prot = explainer_without_prot.model_fairness(protected_variable, privileged_group)
print(fobject_without_prot.fairness_check())

fig = fobject_without_prot.plot(show=False).update_layout(
    autosize=False,
    width=900,
    height=450)
fig.write_image(f"imgs/rf_without_prot.png")
fig.show()

/home/krystian/PythonProjects/eXplainableMachineLearning-2023/.venv/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning:

X does not have valid feature names, but RandomForestClassifier was fitted with feature names



Bias detected in 2 metrics: FPR, STP

Conclusion: your model is not fair because 2 or more criteria exceeded acceptable limits set by epsilon.

Ratios of metrics, based on 'White'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
                TPR       ACC       PPV       FPR       STP
Non-white  0.946032  1.067616  0.987413  0.529412  0.575893
None


## Bias mitigation

In [11]:
protected_variable_train = x_train.race_White.apply(lambda x: "White" if x else "Non-white")


sample_weight = dx.fairness.reweight(
    protected_variable_train, 
    y_train, 
    verbose=False
)

In [12]:
model_reweight = copy(model_without_prot)
model_reweight.fit(x_train_without_prot, y_train, sample_weight=sample_weight)

explainer_reweight = dx.Explainer(
    model_reweight, 
    x_test_without_prot, 
    y_test, 
    label='Random Forest with Reweight mitigation',
    verbose=False
)

fobject_reweight = explainer_reweight.model_fairness(
    protected_variable, 
    privileged_group
)
print(fobject_reweight.fairness_check())

fig = fobject_without_prot.plot([fobject_reweight], show=False).update_layout(
    autosize=False,
    width=900,
    height=450)
fig.write_image(f"imgs/rf_reweight.png")
fig.show()

/tmp/ipykernel_13226/821276099.py:2: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().

/home/krystian/PythonProjects/eXplainableMachineLearning-2023/.venv/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning:

X does not have valid feature names, but RandomForestClassifier was fitted with feature names



Bias detected in 2 metrics: FPR, STP

Conclusion: your model is not fair because 2 or more criteria exceeded acceptable limits set by epsilon.

Ratios of metrics, based on 'White'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
                TPR       ACC       PPV       FPR       STP
Non-white  0.929467  1.063905  0.970833  0.541176  0.577778
None
